# 常微分方程式の周期解の精度保証
$\newcommand{\im}{\mathrm{i}}$
この章では、van der Pol方程式

$$
\frac{d^2 x}{dt^2} - \mu (1-x^2)\frac{dx}{dt} + x = 0.
$$

の周期解の数値検証及び精度保証を行う。

### Newton-Kantorovich を用いた常微分方程式の数値検証

**Newton-Kantrovich 型定理**　
有界線形作用素 $A^\dagger \in \mathcal{L}(X,Y)$, $A \in \mathcal{L} (Y,X)$ を考え、作用素 $F: X \rightarrow Y$ が $C^1$-Fréchet微分可能とする。また $A$ が単射で $AF: X \rightarrow X$ とする。今、$\bar x \in X$に対して、

$$
    \begin{align*}
    \|AF(\bar x)\|_X &\leq Y_0 \\
    ||I - A A^\dagger||_{\mathcal{L}(X)} &\leq Z_0 \\
    ||A (DF(\bar x) - A^\dagger)||_{\mathcal{L}(X)}&\leq Z_1 \\
    ||A (DF(b) - DF(\bar x))||_{\mathcal{L}(X)} &\leq Z_2 (r)r, \quad for \; all \; b \in \overline{B(\bar x, r)}
    \end{align*}
$$

が成り立つとする。このとき、radii polynomialを以下で定義する。

$$
    p(r) := Z_2 (r)r^2 - (1 - Z_1 - Z_0)r + Y_0.
$$

これに対し、$p(r_0)<0$ となる $r_0 > 0$ が存在すれば、$F(\tilde{x}) = 0$ をみたす解 $\tilde x$ が $b \in \overline{B(\bar x, r)}$ 内に一意存在する。

ここで、$DF(\bar x)$ を $F$ の $\bar x$ におけるFréchet微分、$A^\dagger$ を $DF(\bar x)$ の近似、$A$ を $A^\dagger$ の近似左逆作用素とする。($AA^\dagger \approx I$ とするのが一般的である。)

簡易ニュートン写像、banach空間、有界線形作用素、Fréchet微分についても書くべき？？

### van der Pol方程式の周期解を求める

前章と同じ内容なので、簡単に説明する。

最初に、前回用いた関数を呼び出す。

In [1]:
using DifferentialEquations

function vanderpol(du, u , μ ,t)
    x,y = u
    du[1] = y
    du[2] = μ*(1- x ^2)*y - x
end

#a function of  fourier coeffs (lecture7参照)
using FFTW
function odefouriercoeffs(f, I, N, n=1)
    a = I[1]; b = I[2];
    # x_j: equidistance node points
    h = (b-a)/(2N-1)
    j = 0:2N-2
    xⱼ = a .+ j*h
    # f_j: function values on node points
    fⱼ = f(xⱼ)[n,:]
    return (fftshift(fft(fⱼ)))/(2*N-1)
end

using Plots

function plot_solution(u, index) # u = [ω, a_{-N+1}, ..., a_0, ..., a_{N-1}], length(u) = 2N
    # index = 1: profile of solution
    #         2: Fourier mode
    #         3: phase profile
    ω = real(u[1])
    L = 2π / ω
    a = u[2:end]
    N = length(u)/2 # N: size of Fourier
    n_pad = 1000
    a_pad = [zeros(n_pad);a;zeros(n_pad)]
    N_pad = N + n_pad    
    dx = L/(2*N_pad-1)
    x = dx*(0:2*N_pad-2)
    if index == 1
    # Plot profile:
        plot(x,real((2N_pad-1)*ifft(ifftshift(a_pad))),
            xlabel = "\$t\$",
            ylabel = "\$x\\,(t)\$",
            line   = 1.6,
            title  = "Profile of solution",
            size   = (720,400),
            legend = false,
        )
    elseif index == 2
    # Plot Fourier coefficients:
        plot((-N+1):(N-1),abs.(a),yscale=:log10,
            xlabel = "\$k\$",
            ylabel = "\$|a_k\\,|\$",
            line   = 1.6,
            title  = "Fourier coefficients of solution",
            size   = (720,400),
            legend = false,
        )
    elseif index == 3
    # Plot phase:
      k = (-N_pad+1):(N_pad-1)
      plot(real((2N_pad-1)*ifft(ifftshift(a_pad))),real((2N_pad-1)*ifft(ifftshift(a_pad.*(im*k*ω)))),
            xlabel = "\$x(t)\$",
            ylabel = "\$\\dot{x}\\,(t)\$",
            line   = 1.6,
            title  = "Phase plot of a numerical solution",
            size   = (720,400),
            legend = false,
        )
    end
end
function plot_solution!(u)
    L = 2π/real(u[1])
    a = u[2:end]
    N = length(u)/2
    n_pad = 1000
    a_pad = [zeros(n_pad);a;zeros(n_pad)]
    N_pad = N+n_pad
    k = (-N_pad+1):(N_pad-1)
    dx = L/(2*N_pad-1)
    x = dx*(0:2*N_pad-2)
    plot!(real((2N_pad-1)*ifft(ifftshift(a_pad))),real((2N_pad-1)*ifft(ifftshift(a_pad.*(im*k)))),line=1.6,)
end


function powerconvfourier(a::Vector{Complex{T}},p) where T
    M = Int((length(a)+1)/2)
    N = (p-1)*M
    ta = [zeros(N,1);a;zeros(N,1)] # 1. Padding zeros: size(ta) = 2pM-1
    tb = ifft(ifftshift(ta)) # 2. IFFT of ta
    tbᵖ = tb.^p # 3. tb*^tb
    cᵖ = fftshift(fft(tbᵖ))*(2.0*p*M-1)^(p-1)
    return cᵖ[N+1:end-N], cᵖ[p:end-(p-1)]# return (truncated, full) version
end


function F_fourier(x, μ, η₀)
    N = length(x)/2
    ω = x[1]
    a = x[2:end]
    (a³,~) = powerconvfourier(a,3)
    eta = sum(a) - η₀

    k = -(N-1):(N-1)
    f = (- k.^2 * ω^2 - μ* im * k * ω .+ 1) .* a + μ*im * k *ω .* a³ / 3

    return [eta;f]
end


function DF_fourier(x, μ)
    N = Int((length(x))/2)
    ω = x[1]
    a = x[2:end]
    k = (-N+1):(N-1)
    (a³,~) = powerconvfourier(a,3)

    DF = zeros(ComplexF64,2N,2N)

    DF[1,2:end] .= 1
    DF[2:end,1] = (- 2*ω*k.^2 - μ*im*k) .* a + μ*im*k .*a³/3

    (~,a2) = powerconvfourier(a,2)
    
    M = zeros(ComplexF64,2*N-1, 2*N-1)

    for j=(-N+1):(N-1)
        M[k.+N, j+N] = μ*im*k*ω.*a2[k.-j.+(2*N-1)]
    end
    
    L = diagm(- k.^2 * ω^2 - μ* im * k * ω .+ 1)
    
    DF[2:end,2:end] = L + M
    return DF
end



DF_fourier (generic function with 1 method)

次に`DifferentialEquations`を用いて、方程式を解く。

In [2]:
u₀ = [0.0; 2.0]
tspan = (0.0, 300)
μ = 1.0
prob = ODEProblem(vanderpol, u₀, tspan, μ)
sol = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 6121-element Vector{Float64}:
   0.0
   0.009232541136674264
   0.02503848692409142
   0.0435879126511894
   0.0664452197643567
   0.09233090046295041
   0.12139888382691048
   0.15277985837707742
   0.18598700196113963
   0.22027026837640648
   0.25511904177246686
   0.2901157179283844
   0.3250261195014299
   ⋮
 299.3910682814349
 299.4682879337022
 299.5407809249992
 299.6087524958724
 299.6728462014952
 299.733391707013
 299.7906656007293
 299.84478558239294
 299.8961900230025
 299.94618022457564
 299.9938359869561
 300.0
u: 6121-element Vector{Vector{Float64}}:
 [0.0, 2.0]
 [0.01855031657580365, 2.0184626866340247]
 [0.050703595436878524, 2.050027988176535]
 [0.08907287164168692, 2.0869098185480093]
 [0.13728951954725527, 2.1319144129367222]
 [0.1931256500572916, 2.181938353825508]
 [0.2573461918787154, 2.2363355143882697]
 [0.3284087220380286, 2.292086391150688]
 [0.40544112120841447, 2.3465300022922277

次に、大まかな周期を求めて、それを元にフーリエ係数を求める。

In [3]:
#おおよその周期
# a = 30
# b = 36.55
a = 30
app_period = 6.55
timestep = 0.1

f_tmp = sol(a+app_period/2:timestep:a+3*app_period/2)
find_period = abs.(f_tmp .- sol(a))
(~,ind) = findmin(find_period[1,:])
b = a+app_period/2 + timestep*(ind-1)
# abs.(sol(b) .- sol(a))

36.675

In [4]:
#calc fouriercoeffs
N = 61
x = odefouriercoeffs(sol,[a,b],N)

121-element Vector{ComplexF64}:
  0.0001085724317432788 + 1.4473137463048596e-6im
 0.00010856849261749569 + 4.343718755567852e-6im
 0.00010856075337680149 + 7.246219051333433e-6im
 0.00010854882863146494 + 1.015838143239336e-5im
 0.00010853325179996129 + 1.3084515274046183e-5im
 0.00010851317536551067 + 1.6028543855934225e-5im
 0.00010848929273091958 + 1.8994569470070212e-5im
 0.00010846070937696494 + 2.198712809477686e-5im
 0.00010842777510812582 + 2.5010377582766105e-5im
 0.00010839007057935311 + 2.8069262170007842e-5im
  0.0001083473465400478 + 3.116838879521118e-5im
 0.00010829961187444236 + 3.4312967261321555e-5im
 0.00010824619538081315 + 3.750822372000086e-5im
                        ⋮
 0.00010829961187444162 - 3.4312967261320484e-5im
 0.00010834734654004803 - 3.116838879521075e-5im
 0.00010839007057935325 - 2.8069262170007775e-5im
 0.00010842777510813242 - 2.501037758276161e-5im
 0.00010846070937696509 - 2.198712809477703e-5im
 0.00010848929273092907 - 1.8994569470002205e-5im
 

Newton法を反復して、解を得る。

In [5]:
using LinearAlgebra
# Initial value of Newton method
# η₀ = real(sum(x0[2:end]))
η₀ = 0
x = [2*pi/(b-a);x]

# Newton iteration
tol = 5e-10
F = F_fourier(x, μ, η₀)
println("Before step #1, ||F||_1 = $(norm(F,1))")
num_itr = 0

while num_itr ≤ 100
    x = x - DF_fourier(x, μ)\F
    num_itr += 1
    F = F_fourier(x, μ, η₀)
    println("After step #$(num_itr), ||F||_1 = $(norm(F,1))")
    if norm(F,1) < tol
        break
    end
end


Before step #1, ||F||_1 = 17.129706985180142
After step #1, ||F||_1 = 0.16249057553437218
After step #2, ||F||_1 = 0.0009482909730316747
After step #3, ||F||_1 = 1.9840290709432975e-8
After step #4, ||F||_1 = 2.1261384328222529e-13


方程式の周期解を求めることができたところで、Newton-Kantorovich type argumentを用いた数値検証を行う。

#### 作用素$A^\dagger$を定義する

$$
    A^\dagger := 
    \left[\begin{array}{c|ccc}
    DF^{(N)}(\bar x) & \cdots  & {0} & \cdots  \\\hline
    \vdots & \mu_{N+1} & &\Large{0} \\
    \Large{0}& & \mu_{N+2} & \\
    \vdots& \Large{0}& & \ddots
    \end{array}\right]
$$

ここで
$$
    \mu_k := -k^2 \omega^2 - \mu \im k \omega + 1
$$
とする。

#### 作用素 $A$ を定義する

$$
    A := 
    \begin{bmatrix}
    A_N &   &   &  \Large{0} & \\
        & \frac{1}{\mu_{N+1}} & & & \\
        &   & \frac{1}{\mu_{N+2}} & & \\
        &   &   & \frac{1}{\mu_{N+3}} & \\
        & \Large{0} &   &   & \ddots
    \end{bmatrix}
$$

ただし、このときの$A_N$ は ヤコビ行列 $DF^{(N)} (\bar x)$ の近似逆行列 ($A_N \approx DF^{(N)} (\bar x)^{-1}$) であり、数値誤差が生じることに注意する。

#### 各評価を行う

(i)$Y_0$

$$
    ||AF(\bar x)||_x \leq Y_0
$$

(ii)$Z_0$

$$
    ||I - AA^\dagger||_{\mathcal{L}(x)} \leq Z_0
$$

(iii)$Z_1$

$$
    ||A(DF(\bar x) - A^\dagger) h||_x \leq Z_1 , \quad h \in \overline{B(0,1)}
$$

また、ここでいう$\overline{B(0,1)}$とは$||h||_x = 1$ということである。

(iv)$Z_2$
$b \in \overline{B(\bar x, r)}, h \in \overline{B(0,1)}$ として、

$$
    ||A(DF(b) -DF(\bar x)) h||_x \leq Z_2(r)r
$$

以上の(i)から(iv)によって、Newton-Kantorovich type argument による数値検証が可能になる。

では、実際に(i)～(iv)の値を計算する。まず、

\begin{align*}
    A^\dagger &=
    \left[
    \begin{array}{c|cc}
    0 & \cdots & \partial_\alpha \eta & \cdots & 0 & \cdots & 0 \\
    \hline
    \vdots &  & \vdots & & & & \\
    \partial_\omega f_{k}^{(N)} & \cdots & \partial_{a_j} f_{k}^{(N)} & \cdots & & & \\
    \vdots & & \vdots & & \Large{0} & \\
    0 & & & \mu_{N+1} & & & \\
    \vdots & & & & \mu_{N+2} & & \\
    0 & & \Large{0} & & & \ddots
    \end{array}
    \right]=
    \left[
    \begin{array}{c|c}
    0 & A_{a,0}^\dagger \\
    \hline 
    A_{\omega, 1}^\dagger & A_{a,1}^\dagger
    \end{array}
    \right]
\end{align*}

と定義する。またこの行列は $b = (b_0 , b_1) \in \mathbb{C} \times \ell^1_\omega$ に作用するとする。

$A^\dagger$から順に考える。

$$
    \begin{align*}
    A^\dagger b &= 
    \begin{bmatrix}
    0 & A_{a,0}^\dagger \\
    A_{\omega ,1}^\dagger & A_{a,1}^\dagger
    \end{bmatrix}
    \times
    \begin{bmatrix}
    b_0 \\
    b_1
    \end{bmatrix} \\
    &=
    \begin{bmatrix}
    A^\dagger_{a,0} b_1 \\
    A^\dagger_{\omega ,1} b_0 + A^\dagger_{a,1} b_1
    \end{bmatrix}
    =: 
    \begin{bmatrix}
    (A^\dagger b)_0 \\
    (A^\dagger b)_1
    \end{bmatrix}
    \end{align*}
$$

と定義する。またこのとき、$(A^\dagger b)_0$と$(A^\dagger b)_1$はそれぞれ

$$
    \begin{align*}
    (A^\dagger b)_0 = A^\dagger_{a,0} b_1 = \partial_\alpha \eta b^{(N)}_1 \in \mathbb{C} \\
    (A^\dagger b)_1 = A^\dagger_{\omega, 1} b_0 + A^\dagger_{a,1} b_1 \in \mathcal{l}^1_\omega
    \end{align*}
$$

ただし、

$$
    \begin{align*}
    A^\dagger_{\omega,1} &= \partial_\omega f^{(N)}, \\
    (A^\dagger_{a,1})_k &= 
    \begin{cases}
    (\partial_a f^{(N)} b^{(N)}_1)_k \quad (|k| \leq N) \\
    \mu_k (b_1)_k (|k| > N)
    \end{cases}
    \end{align*}
$$

である。次に$A$について考える。

$$
    A^{(N)} = 
    \begin{bmatrix}
    A^{(N)}_{\omega , 0} & A^{(N)}_{a,0} \\
    A^{(N)}_{\omega ,1} & A^{(N)}_{a,1}
    \end{bmatrix}
    \approx DF^{(N)} (\bar x)^{-1} \in \mathbb{C}^{(2N) \times (2N)}??
$$

この行列は数値計算で求めることができる。さらに先程と同様に$b$に作用させると

$$
    \begin{align*}
    (Ab)_0 &= A^{(N)}_{\omega ,0} b_0 + A^{(N)}_{a,0} b^{(N)}_1 \\
    (Ab)_1 &= A^{(N)}_{\omega, 1} b_0 + A_{a,1} b_1
    \end{align*}
$$

ただし、無限次元の$A_{a,1}b_1$は以下のようになる・

$$
    (A_{a,1} b_1 ) _k = 
    \begin{cases}
    (A^{(N)}_{a,1} b^{(N)}_1)_k \quad (|k| \leq N) \\
    \frac{(b_1)_k}{\mu_k} \quad (|k| > N)
    \end{cases}
$$

#### $Y_0$を計算する

$$
    F(\bar x) = (\delta_0 , \delta_1) \in \mathbb{C} \times \mathcal{l}^1_\omega
$$

とすると、$A$の定義より、

$$
    ||AF(\bar x)||_x \leq max\{|A^{(N)}_{\omega, 0} \delta_0 + A^{(N)}_{a,0} \delta^{(N)}_1|, ||A^{(N)}_{\omega,1} \delta_0 + A_{a,1} \delta^{(N)}_1 ||_{\mathcal{l}^1_\omega} + \sum_{|k| > N} \left| \frac{(\delta^{\infty}_1)_k}{\mu_k} \right| \omega_k \}
$$

In [6]:
# start to verify the solution
x[3:2:end] .= 0
ω̄ = real(x[1])
ā = x[2:end]
ν  = 1.1
DF = DF_fourier(x, μ)
A  = inv(DF)
Aₐ₀ = A[1,2:end]
Aₐ₁ = A[2:end,2:end]
Aₒ₁ = A[2:end,1];

入力：N-1, 1, N-1 = 2N-1
出力：2(N-1), N-1, 1, N-1, 2(N-1) = 6(N-1) +1


In [11]:
function F_fourier_ext(x, μ, η₀)
    N = length(x)/2
    ω = x[1]
    a = [zeros(2*(N-1));x[2:end]; zeros(2*(N-1))] #0をどれだけ付け加えるかわからない。
    (a³,~) = powerconvfourier(a,3) # 入力: 2N-1  -> 出力: 2N-1(左), 2*3(N-1)+1(右)
    eta = sum(a) - η₀

    k = -3*N:3*N
    f = (- k.^2 * ω^2 - μ* im * k * ω .+ 1) .* a + μ*im * k *ω .* a³ / 3

    return [eta;f]
end

function wnorm(a, ν)
    m = length(a) # m = 2*N
    N = m/2
    k = -N+1:N-1
    w = ν.^abs.(k)
    return sum(abs.(a).*w)
end

δ  = F_fourier_ext(x, μ, η₀)
δ₀ = δ[1]
δ₁ = δ[2:end]
δ₁_N = δ₁[2*N+1:end-2*N]
δ₁[2*N+1:end-2*N] .= 0
δ₁_tail = δ₁

μₖ(k,ω) = - k.^2 * ω^2 - μ* im * k * ω .+ 1


k_tail = -3*N:3*N

Y₀ = max(abs(A[1,1]*δ₀ + dot(Aₐ₀,δ₁_N)), wnorm(Aₒ₁*δ₀ + Aₐ₁*δ₁_N, ν) + wnorm(δ₁_tail./(abs.(μₖ(k_tail,ω̄))), ν))


println("Y₀ = $Y₀")

LoadError: MethodError: no method matching zeros(::Float64)
[0mClosest candidates are:
[0m  zeros([91m::Union{Integer, AbstractUnitRange}...[39m) at array.jl:498
[0m  zeros([91m::Tuple{Vararg{Union{Integer, AbstractUnitRange}, N} where N}[39m) at array.jl:500
[0m  zeros([91m::Type{StaticArrays.MVector{N, T} where T}[39m) where N at C:\Users\kazuki takahashi\.julia\packages\StaticArrays\uH2MB\src\MVector.jl:25
[0m  ...